In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd

import nibabel as nib
from nimare.transforms import p_to_z

import nilearn
from nilearn import datasets
import nilearn.reporting
from nilearn.image import threshold_img

from nilearn.maskers import NiftiSpheresMasker

In [ ]:
data_dir = "./dset"
age_dir = op.join(data_dir, "age-effect5-21old/habenula")
clust_dir = op.join(data_dir, "age-effect/habenula")

In [ ]:
# get peaks
brik_fn = op.join(age_dir, "sub-group_task-rest_desc-agehabenula_briks+tlrc.BRIK")
table_fn = op.join(age_dir, "sub-group_task-rest_desc-Agehabenula_table.txt")
nii_1s_fn = op.join(age_dir, "sub-group_task-rest_desc-ageDiff_zmap.nii.gz")
cluster_fn = op.join(clust_dir, "clustsim", "clustsim_out.NN3_bisided.1D")

column_names = [".10000", ".05000", ".02000", ".01000"]
cluster_df = pd.read_table(
    cluster_fn, skiprows=8, delim_whitespace=True, names=column_names
)
cluster_df = cluster_df.reset_index()
cluster_df.rename(columns={"index": "pthr"}, inplace=True)

cmap = "cool"
brik_idx = [9]
nii_fns = [nii_1s_fn]
tests = ["1s"]
alpha = ".05000"
pthrs = [0.005]
cohen_thresh = 0

data_df = pd.read_csv(table_fn, sep="\t")
n_sub = data_df.groupby("group").size().sum()
n_sub_1, n_sub_2 = data_df.groupby("group").size().values

for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI \
        -prefix {nii_fn} \
        {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    clusters = nilearn.reporting.get_clusters_table(
        nii_thr_img, z_thresh)
    print(clusters) #coordinates are same as affine of input (MNI)

In [ ]:
# Filter the DataFrame to include only rows where 'Cluster Size (mm3)' is empty
subclusters = clusters[clusters["Cluster Size (mm3)"] == ""]

# Extract the main cluster ID from the 'Cluster ID' column for grouping
subclusters["Main Cluster ID"] = subclusters["Cluster ID"].str.extract(
    r"(\d+)", expand=False
)

# Group by the main cluster ID and extract coordinates
grouped_coordinates = subclusters.groupby("Main Cluster ID").apply(
    lambda df: df[["X", "Y", "Z"]].values.tolist()
)

# Convert the grouped coordinates to a dictionary for easier access
coordinates_dict = grouped_coordinates.to_dict()

# Print the coordinates dictionary
for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")

In [ ]:
# Get a list of all directories in the base directory
dset = os.listdir(data_dir)

# Filter out only subject directories (assuming they start with 'sub-')
sub_dir = [d for d in dset if d.startswith("sub-")]

# Loop over each subject directory
# on hpc will use partipant.tsv
for subject in sub_dir:
    subject_path = os.path.join(data_dir, subject, "func")

    if os.path.isdir(subject_path):
        print(f"Processing subject: {subject}")
        subject_brik = op.join(
            subject_path,
            f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.BRIK",
        )
        subject_nii = op.join(
            subject_path,
            f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.nii.gz",
        )

        convert = f"3dAFNItoNIFTI \
        -prefix {subject_nii} \
        {subject_brik}[{1}]"
        os.system(convert)

In [ ]:
for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")
    for subject in sub_dir:
        subject_path = os.path.join(data_dir, subject, "func")

        if os.path.isdir(subject_path):
            print(f"Processing subject: {subject}")

            subject_nii = nib.load(op.join(
                subject_path,
                f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.nii.gz",
            ))

            mask_img = datasets.load_mni152_brain_mask(resolution=1)

            for coord in coords:
                print(coord)
                masker = NiftiSpheresMasker([coord], radius = 10, mask_img=mask_img)
                masker = masker.fit_transform(subject_nii)
                print(masker)